# GEN level distributions

In [13]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%autoreload 2

In [3]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#test = "/vols/cms/akd116/Offline/output/SM/2019/Mar4_2016_gen/GluGluToHToTauTau_M-125-nospinner_mt_2016.root"
#test = "/vols/build/cms/akd116/newest/run/CMSSW_8_0_25/src/UserCode/ICHiggsTauTau/Analysis/HiggsTauTauRun2/output/VBFHToTauTauUncorrelatedDecay_Filtered_2018_tt_0.root"
#test = "/vols/cms/akd116/Offline/output/SM/2020/Apr26_2018_gen_cpdecays/VBFHToTauTauUncorrelatedDecay_Filtered_tt_2018.root"
#f = uproot.open(test)["gen_ntuple"]
#df = f.pandas.df([
#    "cp_channel",
#    "aco_angle_1",
#    "aco_angle_2",
#    "aco_angle_3",
#    "aco_angle_4",
#    "wt_cp_sm",
#    "wt_cp_ps",
#    "wt_cp_mm",
#],namedecode="utf-8")

df = pd.DataFrame()
for file_number in np.linspace(0, 85, 1):
    test = f"/vols/cms/akd116/Offline/output/SM/2020/Apr26_2018_gen_cpdecays/VBFHToTauTauUncorrelatedDecay_Filtered_2018_tt_{int(file_number)}.root"
    f = uproot.open(test)["gen_ntuple"]
    f.keys()
    df = pd.concat([df, f.pandas.df([
        "cp_channel",
        "aco_angle_1",
        #"aco_angle_2",
        #"aco_angle_3",
        #"aco_angle_4",
        "wt_cp_sm",
        "wt_cp_ps",
        "wt_cp_mm",
    ],namedecode="utf-8")])
df.set_index(["cp_channel"], inplace=True)

In [4]:
df

,aco_angle_1,wt_cp_sm,wt_cp_ps,wt_cp_mm
cp_channel,,,,
-1,9999.0000,1.946667,0.061697,1.223270
-1,9999.0000,1.304017,0.199793,0.379287
-1,9999.0000,1.931258,0.542882,0.926350
2,1.7545,1.838380,0.695684,1.485048
-1,9999.0000,1.247756,1.950832,1.475083
...,...,...,...,...
-1,9999.0000,1.574162,1.523448,1.933410
-1,9999.0000,0.284749,1.707995,1.692921
-1,9999.0000,1.114839,0.655647,0.132585


In [20]:
# options
import dftools
from plotting import var_kw
import itertools
cp_channel = 2
plot_var = "aco_angle_1"
weights = ["wt_cp_sm", "wt_cp_mm", "wt_cp_ps"]
legend_items = [
    r"$\mathcal{CP}\mbox{-}\mathrm{even}\ (\phi_{\tau\tau}=0^{\circ})$",
    r"$\mathcal{CP}\mbox{-}\mathrm{mixed}\ (\phi_{\tau\tau}=45^{\circ})$",
    r"$\mathcal{CP}\mbox{-}\mathrm{odd}\ (\phi_{\tau\tau}=90^{\circ})$",
]

In [21]:
plt.style.use('cms')
prop_cycle = plt.rcParams['axes.prop_cycle']
#colors = itertools.cycle(prop_cycle.by_key()['color'])
colors = itertools.cycle(["#253494","#D62839","#006837"])

fig, ax = plt.subplots(
    2, 1, 
    figsize=(3, 3.4), dpi=200,
    sharex=True, 
    gridspec_kw={"height_ratios": (2, 1), "hspace": 0.1, "wspace": 0.1}
)

def cms_label(ax, label, lumi=35.9, energy=13, extra_label=""):
    ax.text(
        0, 1, r'$\mathbf{CMS}\ \mathit{'+label+'}$',
        ha='left', va='bottom', transform=ax.transAxes,
    )
    ax.text(
        1, 1, r'({:.0f} TeV)'.format(energy),
        ha='right', va='bottom', transform=ax.transAxes,
    )
cms_label(ax[0], "Simulation")

bin_edges = np.linspace(0, 2.*np.pi, endpoint=True, num=51)
bin_low_edge = bin_edges[:-1]
for weight in weights:
    sum_w, _ = np.histogram(
        df.loc[df.eval(f"cp_channel=={cp_channel}"), plot_var],
        bins=bin_edges,
        weights=df.loc[df.eval(f"cp_channel=={cp_channel}"), weight],
    )
    sum_w_total = sum_w.sum()
    sum_w /= sum_w_total
    
    # keep sm to do ratio (it's the first one in the loop anyway)
    if "sm" in weight:
        sum_w_sm = sum_w
    
    sum_ww, _ = np.histogram(
        df.loc[df.eval(f"cp_channel=={cp_channel}"), plot_var],
        bins=bin_edges,
        weights=(df.loc[df.eval(f"cp_channel=={cp_channel}"), weight])**2,
    )
    sum_ww /= (sum_w_total)**2
    
    # keep sm to do ratio (it's the first one in the loop anyway)
    if "sm" in weight:
        sum_ww_sm = sum_ww
    
    color=next(colors),
    ax[0].hist(
        bin_low_edge,
        bins=bin_edges,
        histtype='step',
        weights=sum_w,
        color=color,
    )
    up = sum_w + np.sqrt(sum_ww)
    down = sum_w - np.sqrt(sum_ww)
    ax[0].fill_between(
        bin_edges,
        list(up)+[up[-1]],
        list(down)+[down[-1]],
        step='post',
        alpha=0.2,
        color=color,
    )
    # ratio
    sum_w_ratio = sum_w / sum_w_sm
    sum_ww_ratio = sum_ww / (sum_w_sm**2)
    ax[1].hist(
        bin_low_edge,
        bins=bin_edges,
        histtype='step',
        weights=sum_w_ratio,
        color=color,
    )
    up = sum_w_ratio + np.sqrt(sum_ww_ratio)
    down = sum_w_ratio - np.sqrt(sum_ww_ratio)
    ax[1].fill_between(
        bin_edges,
        list(up)+[up[-1]],
        list(down)+[down[-1]],
        step='post',
        alpha=0.2,
        color=color,
    )
    
ax[0].legend(
    legend_items,
    loc=3, 
    labelspacing=0.2, 
    fontsize=9.,
    framealpha=0,
)

ax[0].set_ylabel("Normalised events")
ax[1].set_ylabel("Ratio")
try:
    ax[1].set_xlabel(var_kw[plot_var])
except KeyError:
    print(f"{plot_var} not defined in var_kw")
    ax[1].set_xlabel(plot_var.replace("_"," "))
ax[1].set_xlim(0, 2.*np.pi)
ax[1].set_xticks([0, np.pi/2, np.pi, 1.5*np.pi, 2.*np.pi])
ax[1].set_xticklabels([0, r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"])

fig.align_labels()
fig.savefig(f"plots/vbf_gen_{plot_var}_cp{cp_channel}.pdf", bbox_inches='tight')